# Task: AP Data - Data Preparation


In [1]:
# Load necessary libraries
library(readxl)
suppressPackageStartupMessages(library(dplyr)) 

In [2]:
working_dir <- getwd()

## 1- Data loading

In [3]:
# Input File path
file_path <- file.path(working_dir, "input", "Data_Preparation_Exercise_2022.xlsx")


In [4]:
# Read the Student Roster sheet into a data frame
rosterDf <- read_excel(file_path, sheet = "Student Roster")

# Read the AP Results sheet into a data frame
apDf <- read_excel(file_path, sheet = "AP Results")

# Read the Teacher sheet into a data frame
teacherDf <- read_excel(file_path, sheet = "Teacher")

## 2- Data Exploration

In [5]:
# Display the first 6 rows of the rosterDf dataframe to quickly inspect its structure and contents
head(rosterDf)

Grade,School,Student ID,Gender,Ethnicity,Lunch Status
<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>
12th Grade,UDC-HS,142021427,F,Black,P
12th Grade,NDHS,200051324,F,Black,F
12th Grade,NDHS,200052033,M,Black,F
12th Grade,NDHS,200273258,M,Black,P
12th Grade,NDHS,200318384,F,Black,P
9th Grade,NDHS,200319929,M,Hispanic,P


In [6]:
# Display the first 6 rows of the apDf dataframe to quickly inspect its structure and contents
head(apDf)

Student Number,Score,Subject
<dbl>,<dbl>,<chr>
303000105,4,Biology
301001086,4,Biology
306001140,2,Biology
303000119,2,Biology
303000023,4,Biology
301001329,3,Biology


In [7]:
# Display the first 6 rows of the teacherDf dataframe to quickly inspect its structure and contents
head(teacherDf)

School,Subject,Teacher
<chr>,<chr>,<chr>
Biology,UDC-HS,Smith
Biology,REH,Morgan
Biology,NDHS,Slaughter
Calc AB,UDC-HS,Ramirez
Calc AB,NDHS,Stenson
Calc BC,UDC-HS,Rametti


#### Check for Data Quality

- **Duplicate check:** Verify that there are no duplicate rows in the datasets.
- **Unique roster rows:** Assume the `roster` dataframe has exactly one row per `Student ID`.
- **Matching student IDs:** Assume every `Student Number` in the AP table has a corresponding entry in the roster table.
- **Column consistency for joins:** Ensure that column names match across dataframes to simplify merging.
- **Missing values:** Ensure there are no missing values in datasets.

In [8]:
# Check for duplicates in rosterDf dataframe
sum(duplicated(rosterDf))

[1] 0

In [9]:
# Check for duplicates in apDf dataframe
sum(duplicated(apDf))

[1] 79

In [10]:
# Check for duplicates in teacherDf dataframe
sum(duplicated(teacherDf))

[1] 0

In [11]:
# Check referential integrity
matches <- apDf$`Student Number` %in% rosterDf$`Student ID`

# 2. 'all(matches)' returns TRUE if every AP student number has a matching roster entry, FALSE if any are missing
all(matches)

[1] TRUE

In [12]:
# Check for duplicate Student IDs in rosterDf
ifelse(length(unique(rosterDf$`Student ID`)) == nrow(rosterDf),"Student ID is unique","Student ID has duplicates")

[1] "Student ID is unique"

In [13]:
# Check for missing values in rosterDf
colSums(is.na(rosterDf))

Grade       School   Student ID       Gender    Ethnicity Lunch Status 
           0            0            0            0            0            0

In [14]:
# Check for missing values in apDf
colSums(is.na(apDf))

Student Number          Score        Subject 
             0              0              0

In [15]:
# Check for missing values in teacherDf
colSums(is.na(teacherDf))

School Subject Teacher 
      0       0       0

#### Data Quality Issues to Address

- **Column consistency for joins:** Rename `Student ID` in rosterDf for `Student Number`.
- **Column naming:** Use the column names specified in the requirements for the output file whenever they exist.
- **Teacher dataset mislabeling:** The `School` column contains `Subject` data, and the `Subject` column contains `School` data. This needs to be corrected.
- **Duplicates:** Remove duplicates from apDf dataframe

## 3- Data cleaning

#### Remove duplicates in apDf dataset.

In [16]:
# Print total number of rows 
print(paste("Total number of rows before removing duplicates", nrow(apDf)))

# Check for duplicate Student Number in AP Results Table 
dup <- apDf %>%
  arrange(`Student Number`)  %>%   # Sort by Student Number
  filter(duplicated(.))                 # Keep only rows where all columns match a previous row

# Write duplicates to a CSV file
dup_output_file_path <- file.path(working_dir, "output", "duplicates.csv")
write.csv(dup, dup_output_file_path, row.names = FALSE)

# Remove duplicates from the original data frame
apDf <- apDf[!duplicated(apDf), ]

# Print total number of rows 
print(paste("Total number of rows is after removing duplicates", nrow(apDf)))


[1] "Total number of rows before removing duplicates 720"
[1] "Total number of rows is after removing duplicates 641"


#### Rename columns consistent with requirements.

In [27]:
# Rename columns in rosterDf dataset.
names(rosterDf) <- c(
    "Grade",
    "High School",
    "Student Number",
    "Gender",
    "Ethnicity",
    "FRPL status")

# Rename columns in teacherDf dataset.
names(teacherDf) <- c("Subject", "High School", "Teacher Name") # Fix incorrect column names in original file

## 4- Final Data Set

In [18]:

# Map grades to graduating years
curr_grad_year <- 2022 # Define current graduation year

grade_to_year <- c(
  "12th Grade" = curr_grad_year,
  "11th Grade" = curr_grad_year + 1,
  "10th Grade" = curr_grad_year + 2,
  "9th Grade"  = curr_grad_year + 3
)

# Prepare final data frame
finalDf <- apDf %>% 
  left_join(rosterDf, by = "Student Number") %>% # Join with roster data to Student Information such as school, grade, demographics
  left_join(teacherDf, by = c("High School", "Subject")) %>% # Join with teacher data to get teacher name
  mutate(
    `Graduating cohort year` = grade_to_year[Grade],   # Map grades to convert grades to graduating year
    `AP Status` = ifelse(Score >= 3, "Pass", "Fail")  # Determine exam pass/fail based of score
  ) %>%
  select(
    `Student Number`,
    `Score`,
    `Subject`,
    `High School`,
    `Graduating cohort year`,
    `AP Status`,
    `FRPL status`,
    `Ethnicity`,
    `Gender`,
    `Teacher Name`)

#### Data Valitation for final dataset.

##### - Missing data

In [19]:
# Check for missing values in finalDf
colSums(is.na(finalDf))

Student Number                  Score                Subject 
                     0                      0                      0 
           High School Graduating cohort year              AP Status 
                     0                      0                      0 
           FRPL status              Ethnicity                 Gender 
                     0                      0                      0 
          Teacher Name 
                    13

In [20]:
# Show rows with missing Teacher Name
finalDf[is.na(finalDf$`Teacher Name`),]


Student Number,Score,Subject,High School,Graduating cohort year,AP Status,FRPL status,Ethnicity,Gender,Teacher Name
<dbl>,<dbl>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>
204010599,2,English Lang and Composition,NDHS,2022,Fail,P,Hispanic,F,NA
270485832,3,English Lang and Composition,NDHS,2022,Pass,P,Black,F,NA
270081946,2,English Lang and Composition,NDHS,2022,Fail,FDC,Hispanic,M,NA
275078996,4,English Lang and Composition,NDHS,2022,Pass,F,Black,M,NA
275078293,2,English Lang and Composition,NDHS,2022,Fail,F,Black,F,NA
270039415,2,English Lang and Composition,NDHS,2022,Fail,FDC,Black,F,NA
215509670,2,English Lang and Composition,NDHS,2022,Fail,FDC,Black,F,NA
203089842,4,English Lang and Composition,NDHS,2022,Pass,F,Black,F,NA
274080092,2,English Lang and Composition,NDHS,2022,Fail,FDC,Black,F,NA


In [21]:
# Check teacherDf dataset to ensure that information about what teacher taught English Lang and Composition at school NDHS is missing

In [22]:
teacherDf[teacherDf$`High School`=="NDHS",]

Subject,High School,Teacher Name
<chr>,<chr>,<chr>
Biology,NDHS,Slaughter
Calc AB,NDHS,Stenson
Environmental Science,NDHS,Reese
US Gov and Politics,NDHS,Amos
US History,NDHS,Lee
World History,NDHS,Monteiro


In [23]:
# Check if some students took same AP test multiple times

In [24]:
finalDf %>%
  group_by(`Student Number`, Subject) %>%
  filter(n() > 1) %>%  # keeps all duplicates
  ungroup() %>%
  arrange(`Student Number`)

Student Number,Score,Subject,High School,Graduating cohort year,AP Status,FRPL status,Ethnicity,Gender,Teacher Name
<dbl>,<dbl>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>
204899629,1,World History,REH,2024,Fail,R,Black,F,Oktani
204899629,2,World History,REH,2024,Fail,R,Black,F,Oktani
206316473,2,World History,REH,2023,Fail,F,Black,F,Oktani
206316473,1,World History,REH,2023,Fail,F,Black,F,Oktani


In [25]:
# Write the final data frame to a CSV file

final_output_file_path <- file.path(working_dir, "output", "final_output.csv")
write.csv(finalDf, final_output_file_path, row.names = FALSE)



## 5 - Observations and Conclusion

- The original AP Results dataset contained duplicates, which were removed. All removed rows were saved to `duplicates.csv`.

 - Inconsistent column names were identified and standardized.

 - Missing teachers was observed Teacher dataset.

 - A lack of clarity was noted in certain column names and definitions. For example:

        - A column indicating whether the student passed the exam. An AP exam score of 3, 4, or 5 is considered passing.
            This column was labeled AP Status with values Pass or Fail, but this may not be consistent with existing dataset from previous years.

        - FRPL (Free or Reduced Price Lunch) status. 
            It is unclear whether this is a transformed field or if same as “Lunch Status” field in the original Student Roster table.

 - Some students have duplicate records for the same AP test. It is unclear whether this is acceptable or if only the highest score should be retained.
 
**Conclusion:** A brief discussion with stakeholders or a better understanging of the merging process may be sufficient to clarify these questions.
